In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 
showHeader('Product Composition Table')

In [ ]:
# fetch data set
df_cas = fh.get_df(os.path.join(hndl.sandbox_dir,'workdf.parquet'))

In [ ]:
df_cas['year'] = df_cas.date.dt.year

In [ ]:
def sort_id(st):
    l = list(st)
    l.sort()
    return l

This interactive table below lists all Tradenanes (from the Open-FF FULL data set) published in the FracFocus bulk download.   

The index of the table is the combination of TradeName, and the set of bgCAS and PercentHigAdditive (the maximum amount of the chemical in the trade-named product) associated with that TradeName in disclosures.  Each unique combination of those three produces a row in the table.

### How to use this table:
Type either a TradeName or CASNumber (or other terms) into the Search bar to filter the list by that search term. You can combine terms in the search to further filter the list. For exmaple, to see the instances of the product Losurf that occurred in 2011, simply use **losurf 2011** in the search bar.  If you want to find all the product names that contained 111-76-2 in 2016, use **111-76-2 2016**.

The table is also sort-able by any column (click a second time for reverse sort).

Note that in many disclosures, entries in the TradeName field are missing or uninformative. Further, some companies **aggregate** many tradenames together, effectively combining all their ingredients into one record in the following table. Those are less useful in understanding what is reported for a given TradeName.



| Column      | Description |
| :----: | :-------- |
|*TradeName*| is a cleaned version of the raw FracFocus text
|*cas_per_tuple*|shows the cluster of chemicals (bgCAS) and their associated PercentHighAdditive for a given TradeName.|   
|*size* |indicates the number of disclosures for this pair of TradeName & cas_per_tuple|
|*year*| years in which this pair occurs|
|*APINumber*| APINumbers for example disclosures with this pair|
|*bgSupplier*| suppliers associated with this pair|


In [ ]:
#df_cas = df_cas[df_cas.data_source=='bulk']
df_cas['cas_per_tuple'] = df_cas.bgCAS.str[:] +' ; ' + df_cas.PercentHighAdditive.astype('str')
df_cas.TradeName = df_cas.TradeName.str.lower()
tn_df = df_cas.groupby(['TradeName','DisclosureId','APINumber','year','bgSupplier'])['cas_per_tuple'].apply(set).reset_index()
tn_df.cas_per_tuple = tn_df.cas_per_tuple.map(lambda x: sort_id(x))
tn_df.cas_per_tuple = tn_df.cas_per_tuple.map(lambda x: th.xlate_to_str(x,'<br>',trunc=False))
gb1 = tn_df.groupby(['TradeName','cas_per_tuple'],as_index=False).size()
gb2 = tn_df.groupby(['TradeName','cas_per_tuple'])['year'].apply(set).reset_index()
gb2.year = gb2.year.map(lambda x: sort_id(x))
gb2.year = gb2.year.map(lambda x: th.xlate_to_str(x,'; ',trunc=False))
mg = pd.merge(gb1,gb2,on=['TradeName','cas_per_tuple'],how='left').reset_index(drop=True)
gb3 = tn_df.groupby(['TradeName','cas_per_tuple'])['APINumber'].apply(set).reset_index()
gb3.APINumber = gb3.APINumber.map(lambda x: sort_id(x))
gb3.APINumber = gb3.APINumber.map(lambda x: th.xlate_to_str(x,' ',totallen=30))
mg = pd.merge(mg,gb3,on=['TradeName','cas_per_tuple'],how='left').reset_index(drop=True)
gb4 = tn_df.groupby(['TradeName','cas_per_tuple'])['bgSupplier'].apply(set).reset_index()
gb4.bgSupplier = gb4.bgSupplier.map(lambda x: sort_id(x))
gb4.bgSupplier = gb4.bgSupplier.map(lambda x: th.xlate_to_str(x,'<br>',totallen=30))
mg = pd.merge(mg,gb4,on=['TradeName','cas_per_tuple'],how='left').reset_index(drop=True)
iShow(mg.sort_values('size',ascending=False),
      maxBytes=0,classes="display compact cell-border")

